### Project 5 Documentation

## Project 5: ARIMA Time Series forecasting

#### 1. Data Source:
- Task 1: NBP dataset, exchange rates bewteen PLN and THB
- Task 2: cryptocurrency exchange rates between USD and BTC


##### 2. Research purpose:

- For both PLN-THB and USD-BTC analysis:
    - Use the ADF-test to test the stationarity of the data
        - Def: A stochastic process is a stationary process, if its joint PDF does not change when shifted in time (if the process does not contain unit root)
        - If a process is stationary, its moments (e.g. mean, variance, skewness, kurtosis) do not change over time
    - Build the ARIMA model with parameters p,d,q; 
        - If the data is stationary: set d=0 (differentiation order)
        - If the data is not stationary, increase the differentiation order until it will be possible to reject the ADF-test H0 (or use a different I model)
    - Find parameters p,q (AR order and moving average window), that will fit the model the best
    - Build the ARIMA model, draw the validation plot to check goodness of fit.
    - If the goodness of fit is not sufficient, use a different integration model (the default I model is shift, however it is possible to use different models, and set the differentiation order d=0: log, exp_decay, sample - rolling mean etc)
    - Use the most optimal (in terms of the ADF-test and currency analysis) Integration model to fit the data
    - Build a different ARIMA model, draw the validation plot
    - Predict the exchange rates based on the fitted ARIMA model
    - Draw conclusions

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from pandas.plotting import register_matplotlib_converters
from datetime import timedelta
register_matplotlib_converters()
import warnings
warnings.simplefilter("ignore")

In [4]:
#confidence level
alpha = 0.05

#The name of the dataset column to analyze
col='Open'

#Select data in range [date_min,date_max]
date_min=pd.to_datetime("2020.01.01")
date_max=pd.to_datetime("2020.06.01")

#Dogecoin dataset (not used in this project)
df_doge="coin_Dogecoin.csv"
#Bitcoin dataset
df_btc="coin_Bitcoin.csv"
#Exchange rates dataset from the NBP
df_currencies="currencies.xls"

In [5]:
# A class CurrencyAnalysis, all the related methods are formalized and 
# joined in a single class

class CurrencyAnalysis:
    def __init__(self,alpha,df_name,col,date_min,date_max,test_days=20,currency_name='',rolling_mean_window=12):
        # confidence level
        self.alpha=alpha
        # dataset
        self.df_name=df_name
        # column to analyze
        self.col=col
        self.currency_name=currency_name
        # The ARIMA model
        self.model=None
        # ARIMA model build results (cross-validation)
        self.results=None
        self.date_min=date_min
        self.date_max=date_max
        # The test data to display is chosen in range [date_max,date_test]
        self.date_test=date_max+timedelta(days=test_days)
        # initialize the dataset
        self.df=self.read_df()
        # Parameter k of the MA model
        self.rwindow=rolling_mean_window
        self.test_days=test_days
        
    @staticmethod
    def init_figure():
        # resize the matplotlib figure
        plt.gcf().set_size_inches(8,3.5)
    
    def _read_df(self,index='Date'):
        # (default) read the BTC dataset (csv)
        df1=pd.read_csv(self.df_name)
        df1[index]=df1[index].apply(pd.to_datetime)
        df1=df1[(df1[index]>=date_min) & (df1[index]<=date_max)]
        df1=df1.set_index(index)
        df1[self.col]=df1[self.col].astype(float)
        return df1
    
    def read_df(self,index='Date'):
        try:
            return self._read_df(index)
        except:
            return self.parse_data2([self.col])
        
    def parse_data2(self,cols):
        # read the NBP dataset (xlsx)
        fname=self.df_name
        df=pd.read_excel(fname)
        df=df.iloc[1:237]
        df['Date']=df['data'].apply(pd.to_datetime)
        for x in cols:
            df[x]=df[x].astype(float)
        df['Open']=df[cols[0]]
        df=df[(df['Date']>=self.date_min) & (df['Date']<=self.date_max)]
        return df
    
    def _test(self,col='Open',index='Date',limits=None):
        # read the test dataset 1 (BTC-USD)
        df_test=pd.read_csv(self.df_name)
        df_test[index]=df_test[index].apply(pd.to_datetime)
        if limits is None:
            df_test=df_test[(df_test[index]>=self.date_max) & (df_test[index]<=self.date_test)]
        else:
            df_test=df_test[(df_test[index]>=limits[0]) & (df_test[index]<=limits[1])]
        df_test=df_test.set_index(index)
        df_test=df_test[self.col].astype(float)
        return df_test
    
    def test(self,col='Open',index='Date',limits=None):
        try:
            return self._test(col,index,limits)
        except:
            # read the test dataset 2 (PLN-THB)
            fname=self.df_name
            df=pd.read_excel(fname)
            df=df.iloc[1:237]
            df['Date']=df['data'].apply(pd.to_datetime)
            df[self.col]=df[self.col].astype(float)
            df['Open']=df[self.col]
            if limits is None:
                df=df[(df['Date']>=self.date_max) & (df['Date']<=self.date_test)]
            else:
                df=df[(df['Date']>=limits[0]) & (df['Date']<=limits[1])]
            
            return df['Open']
        
    def adf_test(self,df=None,col='Open'):
        # Implementation of the ADF-test
        if df is None:
            df=self.df[col]
        result = adfuller(df)
        print(col)
        print('ADF Statistic: {}'.format(result[0]))
        print('p-value: {}'.format(result[1]))
        print('Critical Values:')
        for key, value in result[4].items():
            print('\t{}: {}'.format(key, value))
    
    # Plot the rolling mean and std.
    def get_stationarity(self,timeseries,title=''):
        # rolling statistics (moving average and moving std.)
        rolling_mean = timeseries.rolling(window=self.rwindow).mean()
        rolling_std = timeseries.rolling(window=self.rwindow).std()
        # rolling statistics plot
        original = plt.plot(timeseries, color='blue', label='Original')
        mean = plt.plot(rolling_mean, color='red', label='Rolling Mean')
        std = plt.plot(rolling_std, color='black', label='Rolling Std')
        plt.legend(loc='best',prop={'size': 6})
        plt.title(title)
        plt.show(block=False)

    # Plot the rolling mean and std. && display the ADF-test results
    def stationarity(self,adf_test=True):
        self.init_figure()
        self.get_stationarity(self.df[self.col],self.currency_name)
        # Dickey–Fuller test:
        if adf_test:
            self.adf_test()
    
    # Integration model: df -> df - rolling mean
    def df_log_minus_mean(self,plot=True):
        df_log = self.df[self.col]
        rolling_mean = df_log.rolling(window=12).mean()
        df_log_minus_mean = df_log - rolling_mean
        df_log_minus_mean.dropna(inplace=True)
        if plot:
            self.init_figure()
            self.get_stationarity(df_log_minus_mean,self.currency_name)
            self.adf_test(df_log_minus_mean)
        return df_log_minus_mean
    
    # Integration model: df -> exp(df) with parameters half_life, min_periods
    def df_log_exp_decay(self,half_life,plot=True):
        df_log=self.df[self.col]
        rolling_mean_exp_decay = df_log.ewm(halflife=half_life, min_periods=0, adjust=True).mean()
        df_log_exp_decay = df_log - rolling_mean_exp_decay
        df_log_exp_decay.dropna(inplace=True)
        if plot:
            self.init_figure()
            self.get_stationarity(df_log_exp_decay,self.currency_name)
            self.adf_test(df_log_exp_decay)
        return df_log_exp_decay
    
    # Integration model: df -> df - df.shift()
    def df_log_shift(self,plot=True):
        df_log = self.df[self.col]
        df_log_shift = df_log - df_log.shift()
        df_log_shift.dropna(inplace=True)
        if plot:
            self.init_figure()
            self.get_stationarity(df_log_shift,self.currency_name)
            self.adf_test(df_log_shift)
        return df_log_shift
    
    # Build the ARIMA model with parameters p (autoregressive terms), 
    # d (differentiation order), q (moving average window)
    def arima_build(self,p,d,q,plot=True,limits=None):
        plt.gcf().set_size_inches(4,2.5)
        plt.gca().axes.get_xaxis().set_visible(False)
        df_log=self.df[self.col]
        if limits is not None:
            df_log=df_log[limits[0]:limits[1]]
        #decomposition = seasonal_decompose(df_log)
        self.model = ARIMA(df_log, order=(p,d,q))
        self.results = self.model.fit()
        if plot:
            # shift is the default Integration model
            plt.plot(self.df_log_shift(plot=False))
            plt.plot(self.results.fittedvalues, color='red')
    
    # Predict values based on built ARIMA model
    # ME - mean error between predicted and expected values
    def arima_predict(self,display_me=True):
        results=self.results
        df_log=self.df[self.col]
        predictions_ARIMA_diff = pd.Series(results.fittedvalues, copy=True)
        predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
        predictions_ARIMA_log = pd.Series(df_log.iloc[0], index=df_log.index)
        predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)
        predictions_ARIMA = np.exp(predictions_ARIMA_log)
        #self.init_figure()
        actual=self.test('Open')
        #plt.show()
        predicted=results.predict(1,len(df_log)+self.test_days)
        if isinstance(predicted.index,pd.RangeIndex):
            predicted.index=predicted.index.map(int)
            actual.index=actual.index.map(int)
        predicted=predicted[len(df_log):(len(df_log)+len(actual))]
        plt.plot(predicted,label='Predicted')
        plt.plot(actual,label='Actual')
        if display_me:
            self.calc_me(actual,predicted,skip=0)
        
    #This function is experimental and is not unused in the project
    def predict_iter(self,model_params,window=30,delta=1,lim=np.inf):
        iter_date1=self.date_min
        iter_date2=self.date_min+timedelta(window)
        
        me_list=[]
        predicted_arr=[]
        actual_arr=[]
        i=-1
        while(iter_date2<self.date_max and i<lim-1):
            i+=1
            iter_date1+=timedelta(delta)
            iter_date2+=timedelta(delta)
            p,d,q=model_params
            self.arima_build(p,d,q,plot=False,limits=[iter_date1,iter_date2])
            actual=self.test('Open',limits=[iter_date2,iter_date2+timedelta(delta+1)])
            
            if isinstance(actual.index,pd.DatetimeIndex):
                actual.index=list(range(len(actual.index)))
            
            if len(actual)==0:
                continue
            predicted=self.results.predict(0,actual.index[0]+delta+1)
            if isinstance(actual.index,pd.RangeIndex):
                predicted.index=predicted.index.map(int)
                predicted=predicted[(actual.index[0]-1):(actual.index[0]+delta)]
                
            if len(predicted)==0 or len(actual)==0:
                print(iter_date1)
                print(iter_date2+timedelta(delta))
                continue
            try:
                me=self.calc_me(actual,predicted,skip=0,display=False)
                me_list.append(me)
            except:
                print("AAAA",actual,predicted)
                
            
            predicted_arr.append(predicted.mean())
            actual_arr.append(actual.mean())
        plt.clf()
        plt.plot(predicted_arr,label='Predicted',color='blue')
        plt.plot(actual_arr,label='Actual',color='orange')
        plt.legend()
        plt.show()
        return np.array(me_list)
    
    # Calculate the ME (mean error) between expected and actual values
    def calc_me(self,actual,predicted,skip=0,display=True):
        me=0
        skipped=0
        if isinstance(actual.index,pd.Int64Index):
            for i in actual.index:
                if skipped<skip:
                    skipped+=1
                    continue
                me+=(actual[i]-predicted[i])
        else:
            for i in range(len(actual)):
                if skipped<skip:
                    skipped+=1
                    continue
                me+=(actual[i]-predicted[i])  
        me/=len(actual-skipped)
        if display:
            print("The ME between the currency actual and predicted values is:",me)
        return me
